In [7]:
import re
import os
import glob
import numpy as np
import pandas as pd


infer_dir = "../results/inference"

# task_name = "okvqa"
# ds_split = "train"

task_name = "mmmu"
ds_split = "validation"

# task_name = "mmmu_pro"
# ds_split = "test"


model_names = ["llava-v1.6-vicuna-7b-hf", "llava-v1.6-vicuna-13b-hf",
               "Qwen2.5-VL-7B-Instruct", "InternVL2-8B",
               "deepseek-vl2-tiny", "deepseek-vl2-small"]


def extract_letter(text):
    match = re.search(r"\((\w)\)", text)
    return match.group(1) if match else ""

for mn in model_names:
    data_path = os.path.join(infer_dir, task_name, ds_split, f"{mn}_output.csv")
    data_df = pd.read_csv(data_path)

    arr_path = os.path.join(infer_dir, task_name, ds_split, f"{mn}_prob.npy")
    prob_arr = np.load(arr_path)

    start_chr = 'A'
    choices = []
    for i in range(prob_arr.shape[1]):
        choices.append(start_chr)
        start_chr = chr(ord(start_chr) + 1)

    prob_pred = []
    for i in np.argmax(prob_arr, axis=1):
        prob_pred.append(choices[i])
    prob_pred = np.array(prob_pred, dtype=str)

    generated_outputs = data_df["generated_outputs"].values

    extracted_outputs = []
    for output in generated_outputs:
        pred_txt = str(output)[:10].strip()
        if "\n" in pred_txt:
            pred_txt = pred_txt.split("\n")[1]
        if "(" in pred_txt or ")" in pred_txt:
            pred_txt = extract_letter(pred_txt)
        extracted_outputs.append(pred_txt[:1].upper())
    extracted_outputs = np.array(extracted_outputs)

    errors = data_df["answer"].values.astype(str) == extracted_outputs.astype(str)
    acc = np.mean(errors)

    print(prob_arr.shape)
    # print(mn, acc, np.mean(data_df["answer"].values.astype(str) == prob_pred))
    


(805, 9)
(805, 9)
(805, 9)
(805, 9)
(805, 9)
(805, 9)


In [30]:
prob_arr.shape

(805, 9)

In [35]:
start_chr = 'A'
choices = []
for i in range(prob_arr.shape[1]):
    choices.append(start_chr)
    start_chr = chr(ord(start_chr) + 1)

In [39]:
prob_pred = []
for i in np.argmax(prob_arr, axis=1):
    prob_pred.append(choices[i])
prob_pred = np.array(prob_pred, dtype=str)

In [40]:
prob_pred

array(['A', 'B', 'A', 'C', 'D', 'B', 'C', 'B', 'C', 'A', 'C', 'A', 'D',
       'A', 'D', 'A', 'A', 'A', 'C', 'A', 'D', 'C', 'A', 'C', 'D', 'D',
       'B', 'C', 'A', 'A', 'A', 'E', 'A', 'B', 'B', 'C', 'D', 'D', 'D',
       'B', 'A', 'C', 'D', 'A', 'A', 'B', 'B', 'C', 'D', 'D', 'B', 'B',
       'D', 'A', 'E', 'E', 'B', 'A', 'C', 'A', 'A', 'B', 'B', 'A', 'B',
       'A', 'B', 'A', 'A', 'C', 'B', 'B', 'A', 'B', 'A', 'A', 'A', 'A',
       'A', 'C', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'C', 'D', 'B', 'D',
       'D', 'D', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'B',
       'A', 'A', 'D', 'B', 'A', 'B', 'C', 'C', 'A', 'B', 'B', 'B', 'B',
       'D', 'C', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'C', 'D',
       'D', 'B', 'B', 'D', 'A', 'B', 'C', 'B', 'D', 'B', 'A', 'D', 'D',
       'A', 'B', 'C', 'A', 'A', 'A', 'D', 'C', 'B', 'D', 'B', 'C', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'E', 'A', 'B', 'A', 'B', 'C', 'B',
       'C', 'B', 'B', 'B', 'B', 'A', 'A', 'D', 'A', 'A', 'B', 'C

In [41]:
extracted_outputs == prob_pred

array([ True,  True,  True, False, False,  True, False,  True,  True,
       False,  True,  True, False,  True, False,  True, False, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False, False,  True,  True, False,  True, False, False, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True, False,  True, False, False, False, False,
       False,  True,  True,  True, False,  True, False,  True, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False, False, False,  True,  True,  True,  True, False,  True,
        True,  True, False,  True, False, False,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
       False,  True,